# imports

In [4]:
from pathlib import Path

In [5]:
import bs4

In [6]:
from bs4 import BeautifulSoup as BS

In [7]:
from dataclasses import dataclass

In [8]:
from enum import Enum

In [413]:
from serde import serialize, deserialize
from serde.json import to_json

# Récupération du contenu html

On a **manuellement** sauvegarder les 7 premières pages correspondant aux résultats de la recherche *python* sur [amazon.fr](www.amazon.fr)

In [10]:
racine = Path(".").resolve()
print(racine)

D:\Cours Mecen (M2)\Machine Learning\ML-2023-2024\seance03


In [11]:
for contenu in racine.glob("*"):
    print(contenu)

D:\Cours Mecen (M2)\Machine Learning\ML-2023-2024\seance03\03.ipynb
D:\Cours Mecen (M2)\Machine Learning\ML-2023-2024\seance03\pages


In [12]:
pages = racine / "pages"
print(pages)

D:\Cours Mecen (M2)\Machine Learning\ML-2023-2024\seance03\pages


In [13]:
for chemin in pages.glob("*.html"):
    print(chemin)

D:\Cours Mecen (M2)\Machine Learning\ML-2023-2024\seance03\pages\amazon_p1.html
D:\Cours Mecen (M2)\Machine Learning\ML-2023-2024\seance03\pages\amazon_p2.html
D:\Cours Mecen (M2)\Machine Learning\ML-2023-2024\seance03\pages\amazon_p3.html
D:\Cours Mecen (M2)\Machine Learning\ML-2023-2024\seance03\pages\amazon_p4.html
D:\Cours Mecen (M2)\Machine Learning\ML-2023-2024\seance03\pages\amazon_p5.html
D:\Cours Mecen (M2)\Machine Learning\ML-2023-2024\seance03\pages\amazon_p6.html


In [14]:
htmls = list()
for chemin in pages.glob("*.html"):
    htmls.append(chemin.read_text(encoding="utf8"))

# Exploitation de la première page html

## Exercice

En utilisant les outils de développement du navigateur et `bs4`, récupérer la liste des éléments représentant des livres dans la première page.

In [15]:
soupe = BS(htmls[0])

In [16]:
result_items = soupe.find_all(name="div", attrs={"class": "s-result-item"})

In [17]:
len(result_items)

66

**ATTENTION** on voit visuellement qu'il n'y a que 60 éléments à récupérer dans la page, donc il y a des intrus à identifier.

In [181]:
result_item = result_items[6]

`.prettify()` permet de représenter la page `HTML` bien indentée

In [ ]:
print(result_item.prettify())

In [ ]:
result_item.find_all(name="span", attrs={"class": "a-text-normal"})

In [135]:
result_item.find_all(name="span", attrs={"class": "a-price-whole"})

[<span class="a-price-whole">34,95</span>]

In [182]:
list_author_1 = [
    x.text 
    for x in result_item.select("span.a-size-base:not([class*= 'a-size-base '])") 
    if x.text[0].isupper()
] 

In [184]:
list_author_2 = [
    x.text 
    for x in result_item.find_all(
        name="a", attrs={"class": "a-size-base a-link-normal s-underline-text s-underline-link-text s-link-style"}
    )
    if "Format" not in x.text
]

In [185]:
list_author_1 + list_author_2

['Gérard Swinnen']

In [22]:
def recup_titre_prix(element: bs4.element.Tag) -> tuple[str, str]:
    titre, = element.find_all(name="span", attrs={"class": "a-text-normal"})
    prix, = element.find_all(name="span", attrs={"class": "a-price-whole"})
    return (titre.text, prix.text)
    

In [23]:
recup_titre_prix(result_item)

('Python pour débutants: Programmer un réseau neuronal en 7 jours', '16,99')

In [24]:
extractions = list()
for item in result_items:
    try:
        extractions.append(recup_titre_prix(item))
    except ValueError:
        pass

In [25]:
# il y a normalement 60 livres dans la page
len(extractions)

56

In [26]:
# Inspection visuelle
for extraction in extractions:
    print(extraction)

('Python pour débutants: Programmer un réseau neuronal en 7 jours', '16,99')
("Apprendre\n Python: Un Cours Accéléré sur la Programmation Python et Comment \nCommencer à l’utiliser pour Coder. Apprenez les Bases de Machine \nLearning et de l'analyse de Données", '16,87')
("PYTHON: L'introduction et les fondements pour apprendre à programmer en python en 7 jours", '12,90')
('Python\n GUI Programming with PAGE: Create professional-looking GUIs for Python \napplications efficiently and effectively (English Edition)', '32,69')
('Python pour débutants: Programmer un réseau neuronal en 7 jours', '16,99')
('Apprendre à programmer avec Python 3', '32,40')
("J'apprends à programmer avec Python", '12,95')
("Programmer en Python - Apprendre la programmation de façon claire, concise et efficace - collection O'Reilly", '39,00')
("100\n Exercices Python pour s'entrainer: Un ensemble d'exercices avec \ndifférents niveaux de complexité | Débutant - Intermédiaire - Avancé | \nExercices corrigés pour to

## Exercice

Ajuster la recherche précédente pour récupérer exactement les résultats voulus.

In [27]:
for extraction in extractions:
    if "python" not in extraction[0].lower():
        print(extraction)

('Les algorithmes, c’est plus simple avec un dessin !', '19,90')


In [28]:
class ParsingError(Exception):
    pass

def recup_titre_prix(element: bs4.element.Tag) -> tuple[str, str]:
    try:
        spans = element.find_all(name="span", attrs={"class": "a-text-normal"})
        titre, = spans
        titre = titre.text
    except ValueError:
        raise ParsingError
    try:
        spans = element.find_all(name="span", attrs={"class": "a-price-whole"})
        prix, = spans
        prix = prix.text
    except ValueError:
        raise ParsingError
    if "python" not in titre.lower():
        raise ParsingError
    return (titre, prix)

In [55]:
extractions = list()
for item in result_items:
    try:
        extractions.append(recup_titre_prix(item))
    except ParsingError:
        pass

In [30]:
len(extractions)

55

# Assemblage

In [31]:
class ParsingError(Exception):
    pass

def recup_titre_prix(element: bs4.element.Tag) -> tuple[str, str]:
    try:
        spans = element.find_all(
            name="span", 
            attrs={"class": "a-text-normal"}
        )
        titre, = spans
        titre = titre.text
    except ValueError:
        raise ParsingError
    try:
        spans = element.find_all(
            name="span", 
            attrs={"class": "a-price-whole"}
        )
        prix, = spans
        prix = prix.text
    except ValueError:
        raise ParsingError
    if "python" not in titre.lower():
        raise ParsingError
    return (titre, prix)

In [32]:
extractions = list()
for html in htmls:
    soupe = BS(html, features="lxml")
    result_items = soupe.find_all(
        name="div", 
        attrs={"class": "s-result-item"}
    )
    for item in result_items:
        try:
            extractions.append(recup_titre_prix(item))
        except ParsingError:
            pass

In [42]:
len(extractions)

317

## Exercice

Enrichir le code pour extraire quand c'est possible:
- les auteurs
- l'évalution moyenn
- le nombres d'évaluations
- broché ou relié ou...
- sponsorisé ou pas?

In [411]:
@serialize
@dataclass
class Article:
    titre: str
    prix: float
    auteurs: list
    evaluation: float
    nombre_evaluations: int
    sponsorise: bool

In [402]:
def recupere_titre(element: bs4.element.Tag) -> str:
    try:
        spans = element.find_all(
            name="span", 
            attrs={"class": "a-text-normal"}
        )
        titre, = spans
        titre = titre.text
    except ValueError:
        raise ParsingError
    if "python" not in titre.lower():
        raise ParsingError
    return titre

In [403]:
def recupere_prix(element: bs4.element.Tag) -> str:
    try:
        spans = element.find_all(
            name="span", 
            attrs={"class": "a-price-whole"}
        )
        prix, = spans
        prix = float(prix.text.replace(",", "."))
    except ValueError:
        raise ParsingError
    return prix

In [404]:
def recupere_auteurs(element: bs4.element.Tag) -> list:
    constraints = ["Format", "Relié", "Broché", "Livres audio Audible"]
    list_author_1 = [
        x.text 
        for x in element.select("span.a-size-base:not([class*= 'a-size-base '])") 
        if x.text[0].isupper()
    ]
    list_author_2 = [
        x.text 
        for x in element.find_all(
            name="a", attrs={"class": "a-size-base a-link-normal s-underline-text s-underline-link-text s-link-style"}
        )
        if all(constraint not in x.text for constraint in constraints)
    ]
    auteurs = list_author_1 + list_author_2
    return auteurs

In [405]:
def recupere_evaluation(element: bs4.element.Tag) -> float:
    """a-icon-alt"""
    try:
        spans = element.find_all(
            name="span", 
            attrs={"class": "a-size-base puis-normal-weight-text"}
        )
        rating, = spans
        rating = float(rating.text.replace(",", "."))
    except ValueError:
        raise ParsingError
    return rating

In [406]:
def recupere_nombre_evaluations(element: bs4.element.Tag) -> int:
    try:
        spans = element.find_all(
            name="span", 
            attrs={"class": "a-size-base s-underline-text"}
        )
        nb_rating, = spans
        nb_rating = int(nb_rating.text)
    except ValueError:
        raise ParsingError
    return nb_rating
    

In [407]:
def recupere_sponsorise(element: bs4.element.Tag) -> bool:
    spans = element.find_all(
        name="span",
        attrs={"class": "a-color-base"}
    )
    for span in spans:
        if span.text == "Sponsorisé":
            return True
    return False

In [438]:
def genere_article(element: bs4.element.Tag) -> Article:
    titre = recupere_titre(element)
    prix = recupere_prix(element)
    auteurs = recupere_auteurs(element)
    evaluation = recupere_evaluation(element)
    nombre_evaluations =  recupere_nombre_evaluations(element)
    sponsorise = recupere_sponsorise(element)
    return Article(titre=titre, prix=prix, auteurs=auteurs, evaluation=evaluation, nombre_evaluations=nombre_evaluations,sponsorise=sponsorise)

In [439]:
articles: list[Article] = list()
for html in htmls:
    soupe = BS(html, features="lxml")
    result_items = soupe.find_all(
        name="div", 
        attrs={"class": "s-result-item"}
    )
    for item in result_items:
        try:
            articles.append(genere_article(item)) 
        except ParsingError:
            pass

In [441]:
articles_json = to_json(articles)

In [442]:
file_path = "articles.json"

with open(file_path, "w",  encoding='utf-8') as json_file:
    json_file.write(articles_json)

## Exercice

Sérialiser vers json `articles`.

In [443]:
import polars as pl

In [448]:
print(pl.read_json("articles.json"))

shape: (258, 6)
┌─────────────────────┬────────────┬────────────────────┬───────┬────────────┬─────────────────────┐
│ auteurs             ┆ evaluation ┆ nombre_evaluations ┆ prix  ┆ sponsorise ┆ titre               │
│ ---                 ┆ ---        ┆ ---                ┆ ---   ┆ ---        ┆ ---                 │
│ list[str]           ┆ f64        ┆ i64                ┆ f64   ┆ bool       ┆ str                 │
╞═════════════════════╪════════════╪════════════════════╪═══════╪════════════╪═════════════════════╡
│ ["Violette Chiara", ┆ 4.6        ┆ 65                 ┆ 16.99 ┆ true       ┆ Python pour         │
│ "Benjamin Sp…       ┆            ┆                    ┆       ┆            ┆ débutants:          │
│                     ┆            ┆                    ┆       ┆            ┆ Programme…          │
│ ["Damon Parker"]    ┆ 4.9        ┆ 10                 ┆ 16.87 ┆ true       ┆ Apprendre           │
│                     ┆            ┆                    ┆       ┆          